In [1]:
import numpy as np
import pandas as pd
import pulp
import json
import requests

### CUSTOM
import sys
sys.path.append('../Modules')
import LpPickTeam

In [2]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/' # API URL
resp = requests.get(url)
data = resp.json()
df = pd.DataFrame(data['elements'])[['web_name', 'team', 'element_type', 'ep_next', 'now_cost']]

In [3]:
df = df.rename(columns = {'web_name' : 'Name', 'team' : 'Team', 'now_cost' : 'Cost', 'element_type' : 'PosNum', 'ep_next' : '1_Pts'})

In [4]:
# APPLY ONLY ONCE
position_key = {1 : 'G', 2 : 'D', 3: 'M', 4 : 'F'}
df['Position'] = df['PosNum'].apply(lambda num : position_key[num])
df['Cost'] = df['Cost'].apply(lambda price : price / 10)
df['Team'] = df['Team'].apply(lambda team_num : str(team_num))
df['1_Pts'] = df['1_Pts'].apply(lambda xP : float(xP))
df

,Name,Team,PosNum,1_Pts,Cost,Position
0,Cédric,1,2,2.3,4.5,D
1,Leno,1,1,2.7,4.5,G
2,Xhaka,1,3,2.0,5.0,M
3,Elneny,1,3,1.5,4.5,M
4,Holding,1,2,2.3,4.5,D
...,...,...,...,...,...,...
522,Šarkić,20,1,1.5,4.0,G
523,Gibbs-White,20,3,2.5,5.5,M
524,Mosquera,20,2,1.5,4.0,D
525,Moutinho,20,3,2.0,5.0,M


In [5]:
def position_sort(solution):
    """In-house sorting function for optimal team solution.
    
    solution -- output from AbstractNonsense.pick_team()
    """
    starters = solution[0]['Starters']
    benchs = solution[0]['Bench']
    positions = ['G', 'D', 'M', 'F']
    squad_dict = {'G' : {'Starters' : [], 'Bench' : []}, \
                  'D' : {'Starters' : [], 'Bench' : []}, \
                  'M' : {'Starters' : [], 'Bench' : []}, \
                  'F' : {'Starters' : [], 'Bench' : []}}
    
    for position in positions:
        for starter in starters:
            if pulp.value(starter['var']) == 1.0:
                if starter['position'] == position:
                    squad_dict[position]['Starters'].append([starter['name'], starter['team'], starter['cost'], starter['xP']])
        for bench in benchs:
            if pulp.value(bench['var']) == 1.0:
                if bench['position'] == position:
                    squad_dict[position]['Bench'].append([bench['name'], bench['team'], bench['cost'], bench['xP']])
    
    return squad_dict

In [7]:
# help(LpPickTeam.pick_team)
solution = LpPickTeam.pick_team(df, '1_Pts', (4, 4, 2), 100)
position_sort(solution), solution[1]
# df['1_Pts'].values

({'G': {'Starters': [['Ederson', '13', 5.5, 5.0]],
   'Bench': [['Šarkić', '20', 4.0, 1.5]]},
  'D': {'Starters': [['Van Dijk', '12', 6.5, 4.8],
    ['Robertson', '12', 7.0, 5.2],
    ['Alexander-Arnold', '12', 7.5, 5.5],
    ['Cancelo', '13', 7.0, 4.7]],
   'Bench': [['Kenedy', '6', 4.0, 1.5]]},
  'M': {'Starters': [['Ødegaard', '1', 6.5, 2.8],
    ['Coutinho', '2', 7.0, 2.9],
    ['Rashford', '14', 6.5, 2.8],
    ['Eriksen', '14', 6.5, 2.8]],
   'Bench': [['Devine', '18', 4.5, 1.5]]},
  'F': {'Starters': [['Haaland', '13', 11.5, 5.0], ['Kane', '18', 11.5, 4.5]],
   'Bench': [['Scarlett', '18', 4.5, 1.5]]}},
 45.99999999999999)

In [42]:
df_kiwi = pd.read_csv('../Data/Projected_Points/FPLKiwi_projections.csv')
df_review = pd.read_csv('../Data/Projected_Points/fplreview_projections.csv')
df_official = df

In [67]:
similar = set(df_review['Name'].unique()).intersection(df_kiwi['Name'].unique(), df_official['Name'].unique())
len(similar)

393

In [40]:
set(df_kiwi['Name'].unique()).difference(similar)


{'A Armstrong',
 'A Diallo',
 'A Doucoure',
 'A Gray',
 'A Pereira',
 'A Smith',
 'A Traore',
 'Adarabioyo',
 'Adaramola',
 'Adrian',
 'Ait-Nouri',
 'Ake',
 'Alli',
 'Almiron',
 'Alvarez',
 'Anderson',
 'Andone',
 'Ashby',
 'Augustinsson',
 'B Silva',
 'B Traore',
 'B White',
 'B Williams',
 'Balogun',
 'Batshuayi',
 'Begovic',
 'Bellerin',
 'Ben Johnson',
 'Ben T Davies',
 'Bettinelli',
 'Bogarde',
 'Brennan Johnson',
 'Buendia',
 'Bueno',
 'C Adams',
 'C Doucoure',
 'C Jones',
 'C Wilson',
 'Cafu',
 'Campbell',
 'Canos',
 'Carlos',
 'Cedric',
 'Chesters',
 'Chong',
 'Chrisene',
 'Coventry',
 'Cox',
 'Cutrone',
 'D Gray',
 'D Henderson',
 'D James',
 'D Sanchez',
 'D Ward',
 'Da Costa',
 'Dembele',
 'Dervisoglu',
 'Diaz',
 'Dobbin',
 'Drager',
 'Drameh',
 'E Ferguson',
 'E Martinez',
 'E Mendy',
 'Ekwah',
 'Emerson P',
 'Emerson R',
 'Feeney',
 'Fernandez',
 'Fosu',
 'Francois',
 'Garnacho',
 'Gillespie',
 'Gomes',
 'Guehi',
 'Guimaraes',
 'Gundogan',
 'H White',
 'H Wilson',
 'Hayden

In [38]:
set(df_review['Name'].unique()).difference(similar)

{'A.Armstrong',
 'A.Doucouré',
 'Abdul-Rahman Baba',
 'Adama',
 'Adams',
 'Adrián',
 'Aké',
 'Alejandro Garnacho',
 'Alexander-Arnold',
 'Almirón',
 'Amad Diallo',
 'Andi Zeqiri',
 'Andreas',
 'André Gomes',
 'Ante Palaversa',
 'Arijanet Muric',
 'Arrizabalaga',
 'Arthur Okonkwo',
 'Ashley Young',
 'Axel Tuanzebe',
 'Aït-Nouri',
 'B.Williams',
 'Bech',
 'Begović',
 'Benjamin Mendy',
 'Bergwijn',
 'Bernardo',
 'Bruno Guimarães',
 'Bruno Jordão',
 'Buendía',
 'C.Doucouré',
 'Cafú',
 'Canós',
 'Chalobah',
 'Cheikhou Kouyaté',
 'Clark',
 'Cody Drameh',
 'Connor Ronan',
 'Conor Coventry',
 'Cook',
 'Cédric',
 'Daniel Arzani',
 'Darren Randolph',
 'Darwin',
 'Davies',
 'Davis',
 'De Cordova-Reid',
 'De Gea',
 'Dele',
 'Dembélé',
 'Dennis Praet',
 'Diallo',
 'Diego Carlos',
 'Diogo Jota',
 'Dräger',
 'Emerson',
 'Emerson Royal',
 'Evan Ferguson',
 'Fabio Silva',
 'Ferguson',
 'Fernández',
 'Filip Stevanovic',
 'Fin Stevens',
 'Florin Andone',
 'Frenkie de Jong',
 'Fábio Vieira',
 'Gayle',
 'G

In [72]:
df_official.columns

Index(['Name', 'Team', 'PosNum', '1_Pts', 'Cost', 'Position'], dtype='object')

In [111]:
cleaned_df = pd.DataFrame(columns=['Name', 'Team', 'Position', 'Cost', 'FPL', 'FPLKiwi', 'FPLReview'])
for name in similar:
    official_row = df_official[df_official['Name'] == name]
    kiwi_row = df_kiwi[df_kiwi['Name'] == name]
    review_row = df_review[df_review['Name'] == name]
    new_entry = {'Name' : name, \
                 'Team' : official_row['Team'].values[0], \
                 'Position' : official_row['Position'].values[0], \
                 'Cost' : official_row['Cost'].values[0], \
                 'FPL' : official_row['1_Pts'].values[0], \
                 'FPLKiwi' : kiwi_row['xPts 1'].values[0], \
                 'FPLReview' : review_row['1_Pts'].values[0]}
    new_df = pd.DataFrame(new_entry, index=[0])
    #print(new_df)
    cleaned_df = pd.concat([cleaned_df, new_df], ignore_index=True)
    #df3.reset_index()
cleaned_df['Average'] = 1/3 * (sum(cleaned_df[column] for column in ['FPL', 'FPLKiwi', 'FPLReview']))
#cleaned_df

In [145]:
# zany = cleaned_df[cleaned_df['Average']/cleaned_df['Cost'] > 0.521].sort_values(by='Average', ascending=False)
cleaned_df[cleaned_df['Name'].isin(['Kane', 'Haaland', 'Salah', 'Saka', 'Ederson', 'Alisson'])]

,Name,Team,Position,Cost,FPL,FPLKiwi,FPLReview,Average
80,Saka,1,M,8.0,3.2,3.83,4.2,3.743333
87,Salah,12,M,13.0,5.5,6.28,7.38,6.386667
94,Kane,18,F,11.5,4.5,6.44,6.46,5.8
262,Haaland,13,F,11.5,5.0,4.13,5.42,4.85
309,Ederson,13,G,5.5,5.0,3.83,3.92,4.25
327,Alisson,12,G,5.5,5.5,4.16,4.27,4.643333


In [137]:
for source in ['FPL', 'FPLKiwi', 'FPLReview', 'Average']:
    # help(LpPickTeam.pick_team)
    solution = LpPickTeam.pick_team(zany, source, (4, 4, 2), 100)
    print(source + '\n', position_sort(solution), solution[1], '\n')
    #print(solution[1])
    # df['1_Pts'].values

FPL
 {'G': {'Starters': [['Alisson', '12', 5.5, 5.5]], 'Bench': [['Schmeichel', '10', 5.0, 3.1], ['Pickford', '8', 4.5, 1.7]]}, 'D': {'Starters': [['Robertson', '12', 7.0, 5.2], ['Cancelo', '13', 7.0, 4.7], ['Matip', '12', 6.0, 4.5], ['Dias', '13', 6.0, 4.0], ['Laporte', '13', 6.0, 4.0]], 'Bench': [['Laporte', '13', 6.0, 4.0], ['Mings', '2', 4.5, 2.3]]}, 'M': {'Starters': [['Fabinho', '12', 5.5, 3.5]], 'Bench': []}, 'F': {'Starters': [], 'Bench': []}} 38.9 

FPLKiwi
 {'G': {'Starters': [['Ramsdale', '1', 5.0, 4.07]], 'Bench': [['Schmeichel', '10', 5.0, 3.86]]}, 'D': {'Starters': [['Cancelo', '13', 7.0, 4.84], ['Chilwell', '6', 6.0, 3.99], ['Targett', '15', 5.0, 4.15], ['Digne', '2', 5.0, 3.87], ['Botman', '15', 4.5, 3.83]], 'Bench': [['Walker', '13', 5.0, 2.04]]}, 'M': {'Starters': [['Saint-Maximin', '15', 6.5, 3.97]], 'Bench': []}, 'F': {'Starters': [['Solanke', '3', 6.0, 3.95]], 'Bench': []}} 40.56000000000001 

FPLReview
 {'G': {'Starters': [['Pope', '15', 5.0, 3.98]], 'Bench': [['S

In [148]:
def jaccard_similarity(string1:str, string2:str):
    set1 = set(list(string1))
    set2 = set(list(string2))
    return len(set1.intersection(set2)) / len(set1.union(set2))

In [ ]:
# Non Vectorized
def compare_names(name1:str, name2:str):
    return jaccard_similarity 